# MED-PC Extracting the Recording Data and Metadata

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
from datetime import datetime
from IPython.display import Video
from pathlib import Path

In [2]:
from medpc2excel.medpc_read import medpc_read
import cv2


In [5]:
# setting path
sys.path.append('../../README.mdsrc')

# Getting Tone Times

In [7]:
tone_start_df = pd.read_excel("./data/reward_comp_tone_sleap_frames.xlsx")

In [138]:
tone_start_df["File Name"] = tone_start_df["File Name"].apply(lambda x: x.split(".")[0])

In [140]:
tone_start_df["subj_1"] = tone_start_df["File Name"].apply(lambda x: sorted(x.split("_")[-1].split(".")[0].split("v"))[0].replace("-", ".").strip())

In [141]:
tone_start_df["subj_2"] = tone_start_df["File Name"].apply(lambda x: sorted(x.split("_")[-1].split(".")[0].split("v"))[1].replace("-", ".").strip())

In [142]:
tone_start_df["all_subj"] = tone_start_df.apply(lambda x: tuple(sorted([x["subj_1"], x["subj_2"]])) , axis=1)

In [143]:
tone_start_df = tone_start_df[tone_start_df["Frame Start"].astype(str).str.isnumeric()]

In [144]:
all_itis = [80, 90, 80, 75, 100, 95, 90, 80, 90, 100, 90, 120, 90, 85, 90, 95, 120, 80, 95, 80, 90, 80, 75, 100, 95, 90, 80, 90, 100, 90, 120, 90, 85, 90, 95, 90, 80, 90, 100, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90 ]

In [145]:
all_itis = [num * 30 for num in all_itis]

In [146]:
from functools import reduce

In [147]:
def get_running_total(lst):
    return reduce(lambda acc, x: acc + [acc[-1] + x] if acc else [x], lst, [])

In [148]:
tone_start_df["all_tone_frame"] = tone_start_df.apply(lambda row: get_running_total([row["Frame Start"]] + all_itis), axis=1)

In [149]:
tone_start_df.head()

,File Name,Frame Start,FPS,Strain,subj_1,subj_2,all_tone_frame,all_subj,corner_file,pose_estimation_file
0,10-03-22_Test_1_1-1v1-2,2135,30,C57,1.1,1.2,"[2135, 4535, 7235, 9635, 11885, 14885, 17735, ...","(1.1, 1.2)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
1,10-03-22_Test_2_1-3v1-4,2123,29,C57,1.3,1.4,"[2123, 4523, 7223, 9623, 11873, 14873, 17723, ...","(1.3, 1.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
2,10-03-22_Test_3_2-1v2-2,2125,29,C57,2.1,2.2,"[2125, 4525, 7225, 9625, 11875, 14875, 17725, ...","(2.1, 2.2)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
3,10-03-22_Test_13_2-2v2-4,2138,30,C57,2.2,2.4,"[2138, 4538, 7238, 9638, 11888, 14888, 17738, ...","(2.2, 2.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
4,10-03-22_Test_14_2-1v2-3,2127,29,C57,2.1,2.3,"[2127, 4527, 7227, 9627, 11877, 14877, 17727, ...","(2.1, 2.3)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...


In [150]:
all_corner_h5 = glob.glob("/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/*/*/*.corner.h5")

In [151]:
all_pose_estimation_h5 = glob.glob("/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/h5/*/*/*.h5")

In [152]:
len(all_corner_h5)

33

In [153]:
len(all_pose_estimation_h5)

33

# Calculating the distances

In [154]:
file_name_to_corner = {item.split("/")[-1].split(".")[0].strip(): item for item in all_corner_h5}

In [155]:
file_name_to_pose_estimation = {item.split("/")[-1].split(".")[0].strip(): item for item in all_pose_estimation_h5}

In [156]:
tone_start_df["corner_file"] = tone_start_df["File Name"].map(file_name_to_corner)
tone_start_df["pose_estimation_file"] = tone_start_df["File Name"].map(file_name_to_pose_estimation)

In [176]:
def get_sleap_coordinates(filename):
    """
    """
    with h5py.File(filename, "r") as f:
        dset_names = list(f.keys())
        locations = f["tracks"][:].T
        node_names = [n.decode() for n in f["node_names"][:]]
    return locations, node_names

In [179]:
node_names

['box_bottom_left',
 'reward_port',
 'box_bottom_right',
 'box_top_left',
 'box_top_right']

In [180]:
for index, node in enumerate(node_names):
    tone_start_df["{}_coordinates".format(node)] = tone_start_df["corner_file"].apply(lambda x: get_sleap_coordinates(x)[0][0,index,:,0])

In [181]:
tone_start_df

,File Name,Frame Start,FPS,Strain,subj_1,subj_2,all_tone_frame,all_subj,corner_file,pose_estimation_file,reward_port_coordinates,box_bottom_left_coordinates,box_bottom_right_coordinates,box_top_left_coordinates,box_top_right_coordinates
0,10-03-22_Test_1_1-1v1-2,2135,30,C57,1.1,1.2,"[2135, 4535, 7235, 9635, 11885, 14885, 17735, ...","(1.1, 1.2)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[513.8875473467701, 571.6172631900606]","[390.9854761048566, 550.1045501415538]","[638.3064796862376, 555.5861352215592]","[408.330015339179, 256.61728326956387]","[644.898096304591, 260.851923832358]"
1,10-03-22_Test_2_1-3v1-4,2123,29,C57,1.3,1.4,"[2123, 4523, 7223, 9623, 11873, 14873, 17723, ...","(1.3, 1.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[519.576353919354, 659.5076696013427]","[351.0497840352974, 626.2728051332355]","[693.7994220451658, 629.0949247066953]","[352.9002955462512, 212.30683090705492]","[694.5621623710648, 212.07868790475084]"
2,10-03-22_Test_3_2-1v2-2,2125,29,C57,2.1,2.2,"[2125, 4525, 7225, 9625, 11875, 14875, 17725, ...","(2.1, 2.2)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[508.9149672788628, 621.5982963374584]","[336.35679625338616, 588.6368242233518]","[680.1398630208826, 585.5933059960898]","[343.6821422185993, 166.10302351623739]","[675.7125022249096, 169.20404824713103]"
3,10-03-22_Test_13_2-2v2-4,2138,30,C57,2.2,2.4,"[2138, 4538, 7238, 9638, 11888, 14888, 17738, ...","(2.2, 2.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[512.0763306227065, 569.902165576138]","[389.47144456886895, 548.2629792077062]","[637.5798522460396, 557.7260413970237]","[409.4781326203461, 254.70146395943902]","[643.5017427415908, 262.46446072195494]"
4,10-03-22_Test_14_2-1v2-3,2127,29,C57,2.1,2.3,"[2127, 4527, 7227, 9627, 11877, 14877, 17727, ...","(2.1, 2.3)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[521.0751140419263, 658.3645972713236]","[348.83436667302124, 625.1914641653954]","[692.7881935097188, 628.0072119332935]","[350.74679246264736, 214.02687569988802]","[695.0034616441534, 206.79059001184606]"
5,10-03-22_Test_15_1-2v1-4,2126,29,C57,1.2,1.4,"[2126, 4526, 7226, 9626, 11876, 14876, 17726, ...","(1.2, 1.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[505.4330521173111, 616.7431922367358]","[334.5795884322648, 582.0431856403934]","[674.3358314966611, 585.3479692736961]","[335.7369198234354, 163.69767972487443]","[681.8205431807193, 168.68309070548707]"
6,10-03-22_Test_16_1-1v1-3,2139,30,C57,1.1,1.3,"[2139, 4539, 7239, 9639, 11889, 14889, 17739, ...","(1.1, 1.3)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[515.93901488502, 574.0851976767893]","[389.5177811412061, 544.478349180983]","[636.0045219197326, 552.7090328964389]","[415.5535386121701, 249.525195968645]","[647.132959791255, 263.65911117524]"
7,10-03-22_Test_19_3-2v3-4,2090,30,C57,3.2,3.4,"[2090, 4490, 7190, 9590, 11840, 14840, 17690, ...","(3.2, 3.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[505.09078346236697, 618.8743798488999]","[337.5076307609122, 583.8248584769738]","[677.8345542622602, 589.8278014991329]","[340.68756998880167, 167.2532278005665]","[685.4057773151325, 166.1950963161961]"
8,10-03-22_Test_20_3-1v3-3,2100,30,C57,3.1,3.3,"[2100, 4500, 7200, 9600, 11850, 14850, 17700, ...","(3.1, 3.3)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...,"[515.0328851249129, 571.004000075759]","[389.17302026361233, 541.1946462765961]","[636.1510982595138, 554.0424277591378]","[417.8521836506159, 251.79187150350435]","[648.5186962399791, 261.2638069045993]"
9,10-04

,File Name,Frame Start,FPS,Strain,subj_1,subj_2,all_tone_frame,all_subj,corner_file,pose_estimation_file
0,10-03-22_Test_1_1-1v1-2,2135,30,C57,1.1,1.2,"[2135, 4535, 7235, 9635, 11885, 14885, 17735, ...","(1.1, 1.2)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
1,10-03-22_Test_2_1-3v1-4,2123,29,C57,1.3,1.4,"[2123, 4523, 7223, 9623, 11873, 14873, 17723, ...","(1.3, 1.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
2,10-03-22_Test_3_2-1v2-2,2125,29,C57,2.1,2.2,"[2125, 4525, 7225, 9625, 11875, 14875, 17725, ...","(2.1, 2.2)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
3,10-03-22_Test_13_2-2v2-4,2138,30,C57,2.2,2.4,"[2138, 4538, 7238, 9638, 11888, 14888, 17738, ...","(2.2, 2.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
4,10-03-22_Test_14_2-1v2-3,2127,29,C57,2.1,2.3,"[2127, 4527, 7227, 9627, 11877, 14877, 17727, ...","(2.1, 2.3)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
5,10-03-22_Test_15_1-2v1-4,2126,29,C57,1.2,1.4,"[2126, 4526, 7226, 9626, 11876, 14876, 17726, ...","(1.2, 1.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
6,10-03-22_Test_16_1-1v1-3,2139,30,C57,1.1,1.3,"[2139, 4539, 7239, 9639, 11889, 14889, 17739, ...","(1.1, 1.3)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
7,10-03-22_Test_19_3-2v3-4,2090,30,C57,3.2,3.4,"[2090, 4490, 7190, 9590, 11840, 14840, 17690, ...","(3.2, 3.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
8,10-03-22_Test_20_3-1v3-3,2100,30,C57,3.1,3.3,"[2100, 4500, 7200, 9600, 11850, 14850, 17700, ...","(3.1, 3.3)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...
9,10-04-22_Test_7_3-1v3-4,1838,30,C57,3.1,3.4,"[1838, 4238, 6938, 9338, 11588, 14588, 17438, ...","(3.1, 3.4)",/scratch/back_up/dominance_strain_comparison/p...,/scratch/back_up/dominance_strain_comparison/p...


In [160]:
import h5py
import numpy as np


In [161]:
filename = all_corner_h5[0]

In [162]:

with h5py.File(filename, "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

print("===filename===")
print(filename)
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

===filename===
/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/cd1/day_2/10-04-22_Test_3_5-1v5-4.fixed.corner.h5

===HDF5 datasets===
['edge_inds', 'edge_names', 'instance_scores', 'labels_path', 'node_names', 'point_scores', 'provenance', 'track_names', 'track_occupancy', 'tracking_scores', 'tracks', 'video_ind', 'video_path']

===locations data shape===
(1, 5, 2, 1)

===nodes===
0: box_bottom_left
1: reward_port
2: box_bottom_right
3: box_top_left
4: box_top_right



In [167]:
locations[0,1,0,0]

527.6439208150658

In [168]:
locations[0,1,1,0]

565.2505455907717

In [171]:
for index, row in tone_start_df.iterrows():
    print(row["corner_file"])
    

/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/c57/day_1/10-03-22_Test_1_1-1v1-2.fixed.corner.h5
/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/c57/day_1/10-03-22_Test_2_1-3v1-4.fixed.corner.h5
/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/c57/day_1/10-03-22_Test_3_2-1v2-2.fixed.corner.h5
/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/c57/day_1/10-03-22_Test_13_2-2v2-4.fixed.corner.h5
/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/c57/day_1/10-03-22_Test_14_2-1v2-3.fixed.corner.h5
/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/c57/day_1/10-03-22_Test_15_1-2v1-4.fixed.corner.h5
/scratch/back_up/dominance_strain_comparison/proc/distance_calculation/corner_labeling/c57/day_1/10-03-22_Test_16_1-1v1-3.fixed.corner.h5
/scratch/back_up/dominance_strain_com

# Incorporating Winning and Losing

In [88]:
rc_outcome_df = pd.read_excel("./data/iwata_pilot_3_reward_competition.xlsx")

In [97]:
rc_outcome_df["subj_1"] = rc_outcome_df["Match"].apply(lambda x: sorted([num.strip() for num in x.split("vs")])[0])
rc_outcome_df["subj_2"] = rc_outcome_df["Match"].apply(lambda x: sorted([num.strip() for num in x.split("vs")])[1])
rc_outcome_df["all_subj"] = rc_outcome_df.apply(lambda x: tuple(sorted([x["subj_1"], x["subj_2"]])) , axis=1)

In [98]:
rc_outcome_df

,Date,Cage,Box,Match,Scorer,Trial 1 Winner,Trial 2 Winner,Trial 3 Winner,Trial 4 Winner,Trial 5 Winner,...,Trial 16 Winner,Trial 17 Winner,Trial 18 Winner,Trial 19 Winner,Mouse 1 Wins,Mouse 2 Wins,Ties,subj_1,subj_2,all_subj
0,2022-10-03,1,1,1.1 vs 1.2,Jocelyn,1.1,1.2,1.2,1.1,tie,...,1.2,1.2,1.2,1.2,NaN,NaN,NaN,1.1,1.2,"(1.1, 1.2)"
1,2022-10-03,1,2,1.3 vs 1.4,Jocelyn,1.3,1.4,1.3,1.4,1.4,...,1.3,1.3,1.4,tie,NaN,NaN,NaN,1.3,1.4,"(1.3, 1.4)"
2,2022-10-03,2,3,2.1 vs 2.2,Jocelyn,2.2,2.1,2.1,2.1,2.1,...,2.2,2.1,2.1,2.2,NaN,NaN,NaN,2.1,2.2,"(2.1, 2.2)"
3,2022-10-03,2,4,2.3 vs 2.4,Jocelyn,2.4,2.4,2.4,2.3,2.4,...,2.4,2.4,2.4,2.3,NaN,NaN,NaN,2.3,2.4,"(2.3, 2.4)"
4,2022-10-03,3,1,3.1 vs 3.2,Meghan,3.2,3.1,3.1,3.2,3.2,...,3.1,3.1,3.1,3.1,NaN,NaN,NaN,3.1,3.2,"(3.1, 3.2)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-10-04,3,4,3.1vs3.2,Leo,3.2,3.2,3.2,3.1,3.1,...,3.2,3.2,3.2,3.1,NaN,NaN,NaN,3.1,3.2,"(3.1, 3.2)"
68,2022-10-04,2,1,2.3vs2.4,Aidan,2.4,2.3,2.3,2.3,2.3,...,2.3,2.3,2.3,tie,16.0,2.0,1.0,2.3,2.4,"(2.3, 2.4)"
69,2022-10-04,2,2,2.1vs2.2,Albert,2.2,2.2,2.2,tie,2.2,...,2.2,2.2,2.2,2.2,0.0,17.0,2.0,2.1,2.2,"(2.1, 2.2)"
70,2022-10-04,1,3,1.3vs1.4,Aidan,1.3,1.3,1.4,1.4,1.4,...,1.3,tie,tie,1.3,9.0,6.0,3.0,1.3,1.4,"(1.3, 1.4)"


In [94]:
rc_outcome_df["subj_1"].unique()

array(['1.1', '1.3', '2.1', '2.3', '3.1', '3.3', '4.1', '4.3', '5.1',
       '5.3', '6.1', '6.3', '2.2', '1.2', '4.2', '3.2', '6.2', '5.2'],
      dtype=object)

In [ ]:
rc_outcome_df["subj_1"].unique()